# Import Library

In [1]:
#basic library ===============================================================#=
import pandas as pd                                                           
import numpy as np                                                                                                            
import re                                                                     
import joblib                                                                 
import string                                                                 
#NLTK Corner =================================================================#=
import nltk                                                                    
from nltk.corpus import stopwords                                             
from nltk.tokenize import word_tokenize                                       
from nltk.stem import PorterStemmer                                           
from nltk.tokenize import word_tokenize                                                                                     
#Tensorflow Corner ===========================================================#=
import tensorflow as tf
from sklearn import model_selection 
from sklearn import preprocessing                                                      
from tensorflow.keras.preprocessing.text import Tokenizer                     
from tensorflow.keras.preprocessing.sequence import pad_sequences             
#Sklearn gang                                                                 
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer                   
from sklearn.model_selection import train_test_split                          
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler                   
#Keras club ==================================================================#=
from keras.models import Sequential                                           
from keras import layers                                                      
from tensorflow import keras
from keras.preprocessing import sequence                                      
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D,Bidirectional,MaxPool1D, Flatten
import tensorflow_addons as tfa
#Word2vec
import gensim

In [2]:
X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')
Y = Y.drop("Unnamed: 0", axis='columns')

## Modelling


In [3]:
MAX_NB_WORDS = 256
MAX_SEQUENCE_LENGTH = 150 
EMBEDDING_DIM = 64

In [4]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3497 unique tokens.


In [5]:
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (6236, 150)


## Splitting Train and Test Data

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1)

## Build Model

In [9]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.4))
model.add(Dense(16, activation='softmax'))
model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(), optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), metrics=tf.keras.metrics.top_k_categorical_accuracy) 


In [10]:
#Run model

In [11]:
count = 0
acc = []; loss= []
for item in range(2) :
    history=model.fit(X_train, Y_train,verbose = 1,
               batch_size=32,
               epochs=10,
               validation_data=[X_test, Y_test])
    item = item + 1
    y_pred = model.predict(X_test)
    score = (model.evaluate(X_test, Y_test, batch_size=32, verbose=1))
    acc.append(score[1])
    metric = tfa.metrics.HammingLoss(mode='multilabel', threshold=0.1)
    metric.update_state(Y_test, y_pred)
    loss.append(metric.result().numpy())

    
print(acc)
print(loss)

Epoch 1/10
176/176 [==============================] - 78s 390ms/step - loss: 0.0220 - top_k_categorical_accuracy: 0.7060 - val_loss: 0.0144 - val_top_k_categorical_accuracy: 0.8734
Epoch 2/10
176/176 [==============================] - 63s 360ms/step - loss: 0.0159 - top_k_categorical_accuracy: 0.7557 - val_loss: 0.0143 - val_top_k_categorical_accuracy: 0.8894
Epoch 3/10
176/176 [==============================] - 62s 353ms/step - loss: 0.0151 - top_k_categorical_accuracy: 0.8002 - val_loss: 0.0140 - val_top_k_categorical_accuracy: 0.8638
Epoch 4/10
176/176 [==============================] - 63s 356ms/step - loss: 0.0146 - top_k_categorical_accuracy: 0.8421 - val_loss: 0.0139 - val_top_k_categorical_accuracy: 0.8734
Epoch 5/10
176/176 [==============================] - 62s 351ms/step - loss: 0.0145 - top_k_categorical_accuracy: 0.8592 - val_loss: 0.0139 - val_top_k_categorical_accuracy: 0.8766
Epoch 6/10
176/176 [==============================] - 59s 334ms/step - loss: 0.0143 - top_k_cat

In [31]:
sum(loss)/2

0.1041666641831398

## Model save

In [32]:
# history.save('F_biLSTM_256_128_64_10_a8914_l0.10101162.h5')

## Evaluate

In [33]:
y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test, batch_size=32, verbose=0)
print('Validation accuracy:', score[1])

20/20 [==============================] - 2s 96ms/step
Validation accuracy: 0.8894230723381042


In [34]:
metric = tfa.metrics.HammingLoss(mode='multilabel', threshold=0.1)
metric.update_state(Y_test, y_pred)
print("loss = ", metric.result().numpy())

loss =  0.104467146


In [35]:
sum(loss)/10

0.02083333283662796